In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "dam2411"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style = {"height" : '200px'}),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard CS-340 Dashboard - Dylan Mousseau'))),
    html.Hr(),
    html.Div(
        
    # Filter radio buttons
        dcc.RadioItems(
            id='filter-type',
            options = [
                {'label' : 'Water Rescue', 'value' : 'water'},
                {'label' : 'Mountain/Wilderness Rescue', 'value' : 'mountain'},
                {'label' : 'Disaster Rescue & Individual Tracking', 'value' : 'disaster'},
                {'label' : 'Reset', 'value' : 'reset'}
            ],
            value = 'reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(        
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable = False, # editable flag for table
        filter_action = "native", # table-handled filtering UI
        sort_action = "native", # table-handled sorting UI
        sort_mode = "multi", # multi-column sorting option
        column_selectable = "single", # single column radio button selection
        row_selectable = "single", # single row radio button selection
        row_deletable = False, # delete row parameter set to False
        selected_columns = [], # IDs of selected columns in UI
        selected_rows = [], # IDs of selected rows in UI
        page_action = 'native', # splits rows into traversable pages
        page_current = 0, # current page user is on
        page_size = 15, # row limit per page 
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Filter for rescue scenario
    # Water
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read({
            "animal_type" : 'Dog',
            "breed" : {'$in' : [ 'Labrador Retriever Mix', 'Chesapeak Bay Retriever', 'Newfoundland']},
            "sex_upon_outcome" : 'Intact Female',
            "age_upon_outcome_in_weeks" : {"$gte" : 26.0, "$lte" : 156.0}
        }))
    # Mountain/Wilderness    
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read({
            "animal_type" : 'Dog',
            "breed" : {'$in' : ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            "sex_upon_outcome" : 'Intact Male',
            "age_upon_outcome_in_weeks" : {"$gte" : 26.0, "$lte" : 156.0}
        }))
    # Disaster and Individual Tracking   
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read({
            "animal_type" : 'Dog',
            "breed" : {'$in' : ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            "sex_upon_outcome" : 'Intact Male',
            "age_upon_outcome_in_weeks" : {"$gte" : 20.0, "$lte" : 300.0}
        }))
    # Non-filtered    
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    df.drop(columns=['_id'],inplace=True)    
         
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')])
def update_graphs(viewData, filter_type):
    if filter_type != 'reset':
        df = pd.DataFrame.from_records(viewData)
        return [
            dcc.Graph(            
                figure = px.pie(df, names='breed', title='Preferred Animals')
            )    
        ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('6913beb9bc38274a5bb00fc8'), 'rec_num': 7236, 'age_upon_outcome': '1 year', 'animal_id': 'A689504', 'animal_type': 'Dog', 'breed': 'Australian Kelpie Mix', 'color': 'Black/Tan', 'date_of_birth': '2013-04-05', 'datetime': '2014-10-12 11:33:00', 'monthyear': '2014-10-12T11:33:00', 'name': '*Montie', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.3264922840553, 'location_long': -97.7055240788339, 'age_upon_outcome_in_weeks': 79.3544642857143} 

{'_id': ObjectId('6913beb9bc38274a5bb00fc9'), 'rec_num': 7237, 'age_upon_outcome': '1 year', 'animal_id': 'A690345', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Chocolate', 'date_of_birth': '2013-10-18', 'datetime': '2014-10-31 15:53:00', 'monthyear': '2014-10-31T15:53:00', 'name': 'Pumpkin', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.6961960290177, 'location_long': -97.5145679261, 'age_